<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
    FIT-HCMUS, VNU-HCM 
    <br>
    Introduction To Data Science 
    <br>
    Final project 📌
</div>

<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Preprocessing 📌
</div>

## **1. Set up enviroment**

In [2]:
# !pip install unidecode
# !pip install fuzzywuzzy
import pandas as pd
import numpy as np
import re
import warnings
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

## **2. Reading and cleaning data**

### **2.1. Fbref data**

#### **Raw data inspect**

In [3]:
raw_data_fbref = pd.read_csv("../../data/raw/raw_data_fbref.csv")
raw_data_fbref.head()

,Name,Season,Age,Squad,Country,Comp,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches
0,William Saliba,2018-2019,17.0,Saint-Étienne,fr FRA,1. Ligue 1,4th,16.0,13.0,"1,277",14.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.3,0.3,0.0,0.3,7.0,21.0,4.0,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02,Matches
1,William Saliba,2019-2020,18.0,Saint-Étienne,fr FRA,1. Ligue 1,17th,12.0,11.0,992,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.1,0.3,20.0,33.0,7.0,0.00,0.0,0.00,0.00,0.00,0.02,0.01,0.03,0.02,0.03,Matches
2,William Saliba,2020-2021,19.0,Nice,fr FRA,1. Ligue 1,9th,20.0,20.0,"1,800",20.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.8,0.8,0.0,0.9,36.0,106.0,3.0,0.05,0.0,0.05,0.05,0.05,0.04,0.00,0.04,0.04,0.04,Matches
3,William Saliba,2020-2021,19.0,Arsenal,eng ENG,Jr. PL2 — Div. 1,10th,6.0,6.0,526,5.8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,Matches
4,William Saliba,2021-2022,20.0,Marseille,fr FRA,1. Ligue 1,2nd,36.0,36.0,"3,240",36.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.7,1.7,0.4,2.1,50.0,220.0,14.0,0.00,0.0,0.00,0.00,0.00,0.05,0.01,0.06,0.05,0.06,Matches


**Checking data shape**

In [4]:
fbref_shape = raw_data_fbref.shape

In [5]:
# testing if data include enough 1000 items
print(f"Current shape: {fbref_shape}")

if fbref_shape[0] > 1000:
    print(f"Data is good!")
else:
    print(f"Raw data is absolutely small. Please recheck!")

Current shape: (4687, 37)
Data is good!


**Checking duplicated rows**

In [6]:
index = raw_data_fbref.index
detect_dup_series = index.duplicated(keep='first')
num_duplicated_rows = detect_dup_series.sum()

In [7]:
# checking containing duplicated rows
if num_duplicated_rows == 0:
    print(f"Raw data have no duplicated rows.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate the data.!")

Raw data have no duplicated rows.!


**Column information**

In [8]:
raw_data_fbref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        4687 non-null   object 
 1   Season      4665 non-null   object 
 2   Age         4665 non-null   float64
 3   Squad       4665 non-null   object 
 4   Country     4665 non-null   object 
 5   Comp        4665 non-null   object 
 6   LgRank      4665 non-null   object 
 7   MP          4665 non-null   float64
 8   Starts      4661 non-null   float64
 9   Min         4414 non-null   object 
 10  90s         4415 non-null   float64
 11  Gls         4413 non-null   float64
 12  Ast         4383 non-null   float64
 13  G+A         4383 non-null   float64
 14  G-PK        4411 non-null   float64
 15  PK          4411 non-null   float64
 16  PKatt       4411 non-null   float64
 17  CrdY        4413 non-null   float64
 18  CrdR        4411 non-null   float64
 19  xG          3046 non-null  

It seems there might be an issue with displaying the column names correctly due to duplicates. We'll modify the column names to ensure they present suitable meanings.

In [9]:
new_cols_name = ['Name', 'Season', 'Age', 'Squad', 'Country', 'Comp', 'LgRank', 'MP',
                 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt',
                 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
                 'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'xG/90', 
                 'xAG/90', 'xG+xAG/90', 'npxG/90', 'npxG+xAG/90', 'Matches']

column_name_mapping = dict(zip(raw_data_fbref.columns, new_cols_name))
raw_data_fbref.rename(columns=column_name_mapping, inplace=True)

raw_data_fbref.columns

Index(['Name', 'Season', 'Age', 'Squad', 'Country', 'Comp', 'LgRank', 'MP',
       'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt',
       'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
       'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'xG/90', 'xAG/90',
       'xG+xAG/90', 'npxG/90', 'npxG+xAG/90', 'Matches'],
      dtype='object')

**Field meanings**

| Field            | Description                                                                                                       |
|------------------|-------------------------------------------------------------------------------------------------------------------|
| **Season**       | Season                                                                                                            |
| **Age**          | Age at season start                                                                                               |
| **Country**      | Name of the country                                                                                               |
| **Comp**         | Competition                                                                                                       |
| **LgRank**       | Squad finish in competition                                                                                      |
| **MP**           | Matches Played                                                                                                    |
| **Starts**       | Games that player in starting lineup                                                                                           |
| **Min**          | Minutes played                                                                                                    |
| **90s**          | 90s Played (Minutes played divided by 90)                                                                        |
| **Gls**          | Goals scored or allowed                                                                                           |
| **Ast**          | Assists                                                                                                           |
| **G+A**          | Goals + Assists                                                                                                   |
| **G-PK**         | Non-Penalty Goals                                                                                                 |
| **PK**           | Penalty Kicks Made                                                                                                |
| **PKatt**        | Penalty Kicks Attempted                                                                                           |
| **CrdY**         | Yellow Cards                                                                                                     |
| **CrdR**         | Red Cards                                                                                                        |
| **xG**           | Expected Goals (xG totals include penalty kicks, excluding penalty shootouts)                                   |
| **npxG**         | Non-Penalty Expected Goals                                                                                        |
| **xAG**          | Expected Assisted Goals                                                                                           |
| **npxG+xAG**     | Non-Penalty Expected Goals plus Assisted Goals                                                                    |
| **PrgC**         | Progressive Carries (Carries that move the ball towards the opponent's goal line)                                |
| **PrgP**         | Progressive Passes (Completed passes that move the ball towards the opponent's goal line)                       |
| **PrgR**         | Progressive Passes Received (Completed passes that move the ball towards the opponent's goal line)              |
| **Gls/90**        | Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)                            |
| **Ast/90**        | Assists per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)                          |
| **G+A/90**        | Goals and Assists per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)                 |
| **G-PK/90**       | Non-Penalty Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)                 |
| **G+A-PK/90**       | Non-Penalty Goals plus Assists per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)    |
| **xG/90**         | Expected Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)                   |
| **xAG/90**        | Expected Assisted Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)          |
| **xG+xAG/90**       | Expected Goals plus Assisted Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **npxG/90**       | Non-Penalty Expected Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)        |
| **npxG+xAG/90**   | Non-Penalty Expected Goals plus Assisted Goals per 90 minutes (Minimum 30 minutes played per squad game)           |


**Removing missing season**

In certain football leagues, players may miss a season due to various reasons such as injuries, non-participation, suspensions, transfers, and more. This results in rows with incomplete information in the dataset. Therefore, we are contemplating the removal of these rows.

In [10]:
raw_data_fbref.dropna(subset=['Season'], inplace=True)
raw_data_fbref.shape

(4665, 37)

**Remove unnecessary columns**

In [11]:
raw_data_fbref['Matches'].unique()

array(['Matches', nan], dtype=object)

Column `Matches` only has Matches value. In fact, on website this is a hyperlink to another match so dropping this column is necessary

In [12]:
raw_data_fbref = raw_data_fbref.drop(['Matches'],axis = 1)
raw_data_fbref.columns

Index(['Name', 'Season', 'Age', 'Squad', 'Country', 'Comp', 'LgRank', 'MP',
       'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt',
       'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
       'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'xG/90', 'xAG/90',
       'xG+xAG/90', 'npxG/90', 'npxG+xAG/90'],
      dtype='object')

**Reformating features and Adjusting data types**

- Overall, data type of most fields is correct except for minutes played `Min`, which should be integer instead of object. Here we're going to remove the comma in each value.

In [13]:
def format_min(min):
     if pd.notna(min):
        return int(min.replace(',', ''))
     else:
        return min
    
raw_data_fbref['Min'] = raw_data_fbref['Min'].apply(format_min)

- Additional players personal infos, and fundamental statistics such as `MP`, `Gls`, `Ast`, `G+A`, etc., need to be converted to integers. However, a challenge arises because these columns contain NaN values, which are represented as floating-point numbers, often indicating missing data. To facilitate the conversion of these columns to integers, the transformation from NaN to zero must be performed prior to converting the remaining values.

In [14]:
int_cols = ['Age', 'MP', 'Starts', 'Gls', 'Ast', 'G+A', 
            'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 
            'PrgC', 'PrgP', 'PrgR']

raw_data_fbref[int_cols] = raw_data_fbref[int_cols].fillna(0).astype(int)
raw_data_fbref.dtypes

Name            object
Season          object
Age              int64
Squad           object
Country         object
Comp            object
LgRank          object
MP               int64
Starts           int64
Min            float64
90s            float64
Gls              int64
Ast              int64
G+A              int64
G-PK             int64
PK               int64
PKatt            int64
CrdY             int64
CrdR             int64
xG             float64
npxG           float64
xAG            float64
npxG+xAG       float64
PrgC             int64
PrgP             int64
PrgR             int64
Gls/90         float64
Ast/90         float64
G+A/90         float64
G-PK/90        float64
G+A-PK/90      float64
xG/90          float64
xAG/90         float64
xG+xAG/90      float64
npxG/90        float64
npxG+xAG/90    float64
dtype: object

- Extract the the teams' places in `LgRank` columns.

In [15]:
raw_data_fbref['LgRank'] = raw_data_fbref['LgRank'].str.extract('(\d+)').astype(int)
raw_data_fbref['LgRank'].unique()

array([ 4, 17,  9, 10,  2,  1, 11,  5, 13, 16,  6,  7, 14,  8,  3, 19, 12,
       15, 22, 24, 20, 18, 21, 23, 26, 28, 25])

- Remove redundant letters in values of `Country` column, which in fact is the national flags, causing by crawling process.

In [16]:
# Countries names format and their frequency
count = raw_data_fbref["Country"].value_counts()
count

Country
eng ENG    3267
fr FRA      200
de GER      188
es ESP      179
it ITA      159
nl NED      128
be BEL      101
pt POR       81
sct SCO      55
br BRA       49
dk DEN       38
ar ARG       37
us USA       23
ua UKR       20
at AUT       19
se SWE       16
cz CZE       13
ch SUI       12
tr TUR       11
gr GRE       11
co COL       10
no NOR        9
jp JPN        8
uy URU        5
rs SRB        4
hr CRO        4
py PAR        4
ec ECU        3
pl POL        3
mx MEX        3
sa KSA        2
ru RUS        1
kr KOR        1
za RSA        1
Name: count, dtype: int64

In [17]:
# get rid of the first word of countries name
raw_data_fbref['Country'] = raw_data_fbref['Country'].str.split().str[1]
count = raw_data_fbref["Country"].value_counts()
count

Country
ENG    3267
FRA     200
GER     188
ESP     179
ITA     159
NED     128
BEL     101
POR      81
SCO      55
BRA      49
DEN      38
ARG      37
USA      23
UKR      20
AUT      19
SWE      16
CZE      13
SUI      12
TUR      11
GRE      11
COL      10
NOR       9
JPN       8
URU       5
SRB       4
CRO       4
PAR       4
ECU       3
POL       3
MEX       3
KSA       2
RUS       1
KOR       1
RSA       1
Name: count, dtype: int64

- Data of players currently playing in premier league may include their previous records from other leagues in which `Season` format is different. It is neccessary to adjust this for later stages.

In [18]:
season_count = raw_data_fbref['Season'].value_counts()
print(season_count)
print(f"total: {season_count.sum()}")

Season
2022-2023    674
2023-2024    640
2021-2022    627
2020-2021    585
2019-2020    549
2018-2019    473
2017-2018    416
2016-2017    325
2015-2016    173
2014-2015     63
2021          21
2017          19
2016          18
2022          18
2019          17
2018          16
2020          15
2023          10
2015           6
Name: count, dtype: int64
total: 4665


In [19]:
def format_season(season):
    if pd.notna(season) and '-' in season:
        return season
    elif pd.notna(season):
        return f"{season}-{int(season) + 1}"
    else:
        return season
raw_data_fbref['Season'] = raw_data_fbref['Season'].apply(format_season)

In [20]:
season_count = raw_data_fbref['Season'].value_counts()
print(season_count)
print(f"total: {season_count.sum()}")

Season
2022-2023    692
2023-2024    650
2021-2022    648
2020-2021    600
2019-2020    566
2018-2019    489
2017-2018    435
2016-2017    343
2015-2016    179
2014-2015     63
Name: count, dtype: int64
total: 4665


- Remove prefix number, and normalize league name in `Comp` column using `unicode` library.

In [21]:
raw_data_fbref['Comp'] = raw_data_fbref['Comp'].apply(unidecode)
raw_data_fbref['Comp'] = raw_data_fbref['Comp'].str.replace(r'^(\d+\.?\s*)+', '', regex=True)
comp_counts = raw_data_fbref['Comp'].value_counts()
comp_counts

Comp
Premier League        2131
Championship           566
Jr. PL2 -- Div. 1      283
Serie A                197
Bundesliga             187
Ligue 1                172
La Liga                161
Jr. PL2 -- Div. 2      115
League One             108
Eredivisie             100
Primeira Liga           81
First Division A        76
Premiership             49
League Two              44
Primera Div             41
Superliga               38
Jr. PL2                 35
Eerste Divisie          28
Ligue 2                 28
MLS                     19
Segunda Division        18
Pro League A            14
Serie B                 14
First League            13
Jr. U19 Bundesliga      13
Allsvenskan             13
Super Lg                12
Super Lig               11
Super League            11
National League         11
Primera A               10
J1 League                7
Pro League               7
Tippeligaen              5
Primera Division         5
SuperLiga                4
First Division B       

**Processed data**

In [22]:
processed_data_fbref = raw_data_fbref
processed_data_fbref.head()

,Name,Season,Age,Squad,Country,Comp,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls/90,Ast/90,G+A/90,G-PK/90,G+A-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90
0,William Saliba,2018-2019,17,Saint-Étienne,FRA,Ligue 1,4,16,13,1277.0,14.2,0,0,0,0,0,0,1,0,0.3,0.3,0.0,0.3,7,21,4,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02
1,William Saliba,2019-2020,18,Saint-Étienne,FRA,Ligue 1,17,12,11,992.0,11.0,0,0,0,0,0,0,0,0,0.2,0.2,0.1,0.3,20,33,7,0.00,0.0,0.00,0.00,0.00,0.02,0.01,0.03,0.02,0.03
2,William Saliba,2020-2021,19,Nice,FRA,Ligue 1,9,20,20,1800.0,20.0,1,0,1,1,0,0,2,0,0.8,0.8,0.0,0.9,36,106,3,0.05,0.0,0.05,0.05,0.05,0.04,0.00,0.04,0.04,0.04
3,William Saliba,2020-2021,19,Arsenal,ENG,Jr. PL2 -- Div. 1,10,6,6,526.0,5.8,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,0,0,0,0.00,0.0,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN
4,William Saliba,2021-2022,20,Marseille,FRA,Ligue 1,2,36,36,3240.0,36.0,0,0,0,0,0,0,5,0,1.7,1.7,0.4,2.1,50,220,14,0.00,0.0,0.00,0.00,0.00,0.05,0.01,0.06,0.05,0.06


### **2.2. Transfermarkt data**

#### **Raw data inspect**

In [24]:
raw_data_trans = pd.read_csv("../../data/raw/raw_data_trans.csv")
raw_data_trans.head()

,Name,Season,Injury,from,until,Days,Games missed
0,Ederson,20/21,Virus,"Dec 27, 2020","Jan 12, 2021",16 days,3
1,Ederson,19/20,Ill,"Dec 30, 2019","Jan 7, 2020",8 days,3
2,Ederson,19/20,Ill,"Dec 8, 2019","Dec 14, 2019",6 days,1
3,Ederson,19/20,muscular problems,"Nov 6, 2019","Nov 22, 2019",16 days,4
4,Ederson,17/18,Facial injury,"Sep 10, 2017","Sep 12, 2017",2 days,-


It seems that, for this data table, the data is relatively clean, and there are no columns containing junk data. Therefore, removing columns is not yet necessary at the current time. However, some reformation must be done for synchronization with the previous data.

**Checking data shape**

In [27]:
trans_shape = raw_data_trans.shape
trans_shape

(3233, 7)

In [28]:
# testing if data include enough 1000 items
print(f"Current shape: {fbref_shape}")

if fbref_shape[0] > 1000:
    print(f"Data is good!")
else:
    print(f"Raw data is absolutely small. Please recheck!")

Current shape: (4687, 37)
Data is good!


**Checking duplicated rows**

In [30]:
index = raw_data_trans.index
detect_dup_series = index.duplicated(keep='first')
num_duplicated_rows = detect_dup_series.sum()

In [31]:
# checking containing duplicated rows
if num_duplicated_rows == 0:
    print(f"Raw data have no duplicated rows.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate the data.!")

Raw data have no duplicated rows.!


**Column informations**

In [32]:
raw_data_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          3233 non-null   object
 1   Season        3233 non-null   object
 2   Injury        3233 non-null   object
 3   from          3233 non-null   object
 4   until         3233 non-null   object
 5   Days          3233 non-null   object
 6   Games missed  3233 non-null   object
dtypes: object(7)
memory usage: 176.9+ KB


The dataset appears to be well-maintained with no discernible issues, and there is no necessity to eliminate any columns or rows.

**Field meanings**

| Field           | Description                                                  |
|-----------------|--------------------------------------------------------------|
| **Name**        | The name of the player.                                      |
| **Season**      | The season during which the injury occurred.                 |
| **Injury**      | The type or nature of the injury sustained by the player.    |
| **From**        | The date when the player suffered the injury.                |
| **Until**       | The date when the player recovered from the injury.          |
| **Days**        | The duration of the injury in days.                          |
| **Games missed**| The number of games the player missed due to the injury.     |


**Reformating features and Adjusting data types**

- We need to change `Season` format to match the one of fbref data for later stage.

In [33]:
seas_count = raw_data_trans['Season'].value_counts()
print(seas_count)
print(f"Total: {seas_count.sum()}")

Season
20/21    598
21/22    508
19/20    474
18/19    460
22/23    414
17/18    201
23/24    165
16/17    156
15/16    116
14/15     85
13/14     56
Name: count, dtype: int64
Total: 3233


In [34]:
def format_trans_season (season):
    if pd.notna(season):
        year = season.split('/')
        return f"20{year[0]}-20{year[1]}"
    else:
        return season
raw_data_trans['Season'] = raw_data_trans['Season'].apply(format_trans_season)
seas_count = raw_data_trans['Season'].value_counts()
print(seas_count)
print(f"Total: {seas_count.sum()}")

Season
2020-2021    598
2021-2022    508
2019-2020    474
2018-2019    460
2022-2023    414
2017-2018    201
2023-2024    165
2016-2017    156
2015-2016    116
2014-2015     85
2013-2014     56
Name: count, dtype: int64
Total: 3233


- Adjusting `Days` format to int, and removing unnecessary 'days' string.

In [35]:
raw_data_trans  = raw_data_trans.copy()
raw_data_trans['Days'] = raw_data_trans['Days'].str.split().str[0].astype('int64')
raw_data_trans['Days'].dtypes

dtype('int64')

**Processed data**

In [36]:
processed_data_trans = raw_data_trans
processed_data_trans.head()

,Name,Season,Injury,from,until,Days,Games missed
0,Ederson,2020-2021,Virus,"Dec 27, 2020","Jan 12, 2021",16,3
1,Ederson,2019-2020,Ill,"Dec 30, 2019","Jan 7, 2020",8,3
2,Ederson,2019-2020,Ill,"Dec 8, 2019","Dec 14, 2019",6,1
3,Ederson,2019-2020,muscular problems,"Nov 6, 2019","Nov 22, 2019",16,4
4,Ederson,2017-2018,Facial injury,"Sep 10, 2017","Sep 12, 2017",2,-


## 3. Merging data

**Merging 2 dataframes can help to attach additional informations for later stage**

There is a challenge where the names of players in two dataframes may not always match. Therefore, we require the assistance of a package called `fuzzywuzzy` to assess the similarity between two strings for merging purposes.

In [37]:
# create a dictionary for matching names of 2 Name columns of 2 dataframes
name_matches = {}
fbref_name = processed_data_fbref['Name'].unique()
trans_name = list(processed_data_trans['Name'].unique())

for name in fbref_name:
    if trans_name:
        best_match, score = process.extractOne(name, trans_name,scorer = fuzz.token_set_ratio)
        if score > 90:
            name_matches[name] = best_match
            trans_name.remove(best_match)
        else:
            name_matches[name] = name
    else:
        name_matches[name] = name


processed_data_fbref['Name'] = processed_data_fbref['Name'].map(name_matches)

# Merge based on both 'Name' and 'Season'
merged_df = pd.merge(processed_data_fbref, processed_data_trans , left_on=['Name', 'Season'], right_on=['Name', 'Season'], how='left')

In [38]:
# matching names found
pd.Series(name_matches)

William Saliba         William Saliba
Declan Rice               Declan Rice
Bukayo Saka               Bukayo Saka
Ben White                   Ben White
Martin Ødegaard       Martin Ødegaard
                           ...       
Jack Cork                   Jack Cork
Michael Obafemi       Michael Obafemi
Hjalmar Ekdal           Hjalmar Ekdal
Han-Noah Massengo    HanNoah Massengo
Arijanet Muric         Arijanet Muric
Length: 536, dtype: object

In [40]:
merged_df

,Name,Season,Age,Squad,Country,Comp,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls/90,Ast/90,G+A/90,G-PK/90,G+A-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90,Injury,from,until,Days,Games missed
0,William Saliba,2018-2019,17,Saint-Étienne,FRA,Ligue 1,4,16,13,1277.0,14.2,0,0,0,0,0,0,1,0,0.3,0.3,0.0,0.3,7,21,4,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02,Hamstring injury,"May 23, 2019","Sep 28, 2019",128.0,22
1,William Saliba,2019-2020,18,Saint-Étienne,FRA,Ligue 1,17,12,11,992.0,11.0,0,0,0,0,0,0,0,0,0.2,0.2,0.1,0.3,20,33,7,0.00,0.0,0.00,0.00,0.00,0.02,0.01,0.03,0.02,0.03,Metatarsal fracture,"Nov 4, 2019","Jan 20, 2020",77.0,19
2,William Saliba,2020-2021,19,Nice,FRA,Ligue 1,9,20,20,1800.0,20.0,1,0,1,1,0,0,2,0,0.8,0.8,0.0,0.9,36,106,3,0.05,0.0,0.05,0.05,0.05,0.04,0.00,0.04,0.04,0.04,NaN,NaN,NaN,NaN,NaN
3,William Saliba,2020-2021,19,Arsenal,ENG,Jr. PL2 -- Div. 1,10,6,6,526.0,5.8,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,0,0,0,0.00,0.0,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,William Saliba,2021-2022,20,Marseille,FRA,Ligue 1,2,36,36,3240.0,36.0,0,0,0,0,0,0,5,0,1.7,1.7,0.4,2.1,50,220,14,0.00,0.0,0.00,0.00,0.00,0.05,0.01,0.06,0.05,0.06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,Arijanet Muric,2020-2021,21,Girona,ESP,Segunda Division,5,2,2,180.0,2.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Muscle injury,"Oct 20, 2020","Nov 10, 2020",21.0,6
6003,Arijanet Muric,2020-2021,21,Willem II,NED,Eredivisie,14,14,14,1260.0,14.0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Muscle injury,"Oct 20, 2020","Nov 10, 2020",21.0,6
6004,Arijanet Muric,2021-2022,22,Adana Demirspor,TUR,Super Lig,9,31,31,2778.0,30.9,0,0,0,0,0,0,3,0,NaN,NaN,NaN,NaN,0,0,0,0.00,0.0,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,Head injury,"Feb 6, 2022","Feb 11, 2022",5.0,2
6005,Arijanet Muric,2022-2023,23,Burnley,ENG,Championship,1,41,41,3614.0,40.2,0,0,0,0,0,0,3,0,0.0,0.0,0.2,0.2,0,6,0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN


**Check if merging is correct**

In [42]:
assert len(processed_data_fbref['Name'].unique()) == len(name_matches.keys())
assert len(merged_df['Season'].unique()) == len(processed_data_fbref['Season'].unique())

## 4. Save processed data

In [43]:
# save data to processed data folder
merged_df.to_csv('../../data/processed/processed_fb_data.csv', index=False, encoding='utf-8')